In [6]:
import pandas as pd
import sqlite3

In [7]:
#load in the hospital dataset
df = pd.read_csv('Hospital_patients_datasets.csv')

In [8]:
#check the dataframe
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589980e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262960e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679510e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841190e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [9]:
#check the info about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [10]:
#check the shape of the dataset
df.shape

(110527, 14)

In [11]:
#check for null values and sum any
df.isna().sum()

PatientId         0
AppointmentID     0
Gender            0
ScheduledDay      0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64

In [12]:
#check for duplicates and some across all columns
df[:].duplicated().sum()

0

In [13]:
#convert the datatype of day columns to datetime
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay']).dt.date.astype('datetime64[ns]')
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay']).dt.date.astype('datetime64[ns]')

In [14]:
#rename some particular columns
df = df.rename( columns={'Hipertension': 'Hypertension',
                         'Handcap': 'Handicap',
                         'SMS_received': 'SMSRecevied',
                         'No-show': 'NoShow'})

In [15]:
# drop columns that are not of interest in the dataframe
df.drop(['PatientId'], axis=1, inplace=True)
df.drop(['AppointmentID'], axis=1, inplace=True)
df.drop(['Neighbourhood'], axis=1, inplace=True)

In [16]:
# drop age rows with 0 values
df.drop(df[df['Age'] == 0].index, inplace=True)

In [17]:
# Generating labels for age intervals (e.g., '1 - 20', '21 - 40', etc.)
labels = ["{0} - {1}".format(i, i + 20) for i in range(1, 118, 20)]

#create the bins
bins = range(1, 130, 20)

# use the cut function to categorize the age into groups
df['Age_group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)    

In [18]:
# drop the Age column
df.drop('Age', axis=1, inplace=True)

In [19]:
# convert 'NoShow' values into binary values (1 for 'Yes' and 0 for 'No').
df['NoShow'] = df['NoShow'].map({'Yes':1, 'No':0})

In [20]:
# check the cleaned dataframe
df.head()

,Gender,ScheduledDay,AppointmentDay,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSRecevied,NoShow,Age_group
0,F,2016-04-29,2016-04-29,0,1,0,0,0,0,0,61 - 81
1,M,2016-04-29,2016-04-29,0,0,0,0,0,0,0,41 - 61
2,F,2016-04-29,2016-04-29,0,0,0,0,0,0,0,61 - 81
3,F,2016-04-29,2016-04-29,0,0,0,0,0,0,0,1 - 21
4,F,2016-04-29,2016-04-29,0,1,1,0,0,0,0,41 - 61


In [21]:
df['Age_group'].value_counts()

Age_group
41 - 61      30081
21 - 41      28835
1 - 21       28309
61 - 81      16910
81 - 101      2845
101 - 121        7
Name: count, dtype: int64

In [61]:
df['Gender'].value_counts()

Gender
F    70119
M    36869
Name: count, dtype: int64

In [22]:
df['NoShow'].value_counts()

NoShow
0    85308
1    21680
Name: count, dtype: int64

In [ ]:
# export the cleaned dataset and set the index=false
df.to_csv('patients.csv', index=False) 

In [2]:
import sqlite3

In [3]:
cnn = sqlite3.connect('jupyter_sql_tutorial.db')

In [23]:
df.to_sql('patients', cnn)

106988

In [29]:
%load_ext sql

In [30]:
%sql sqlite:///jupyter_sql_tutorial.db

TASK 1: How many values are there in the given dataset?

In [32]:
%%sql

SELECT COUNT(*) as count 
FROM patients

 * sqlite:///jupyter_sql_tutorial.db
Done.


count
106988


TASK 2: Count the number of appointments for each day in the given dataset:

In [35]:
%%sql

SELECT 
            AppointmentDAY as appointmentday, count(AppointmentDAY) as total_appointment
FROM 
            patients
GROUP BY 
            1

 * sqlite:///jupyter_sql_tutorial.db
Done.


appointmentday,total_appointment
2016-04-29 00:00:00,3104
2016-05-02 00:00:00,4214
2016-05-03 00:00:00,4129
2016-05-04 00:00:00,4048
2016-05-05 00:00:00,4113
2016-05-06 00:00:00,3791
2016-05-09 00:00:00,4352
2016-05-10 00:00:00,4177
2016-05-11 00:00:00,4347
2016-05-12 00:00:00,4233


TASK 3: Calculate the average number of appointments(Set to nearest whole number) per day in the given dataset.

In [38]:
%%sql

SELECT 
        ROUND(AVG(T1.count),0) as average_appointment

FROM 
        (SELECT 
                AppointmentDAY as appointmentday, COUNT(AppointmentDAY) as count
        FROM 
                patients
        GROUP BY 
                1) as T1

 * sqlite:///jupyter_sql_tutorial.db
Done.


average_appointment
3963.0


TASK 4: Find the day with the highest number of appointments in the given dataset.

In [40]:
%%sql

SELECT 
        AppointmentDAY, COUNT(AppointmentDAY) as number_of_appointments
FROM 
        patients
GROUP BY 
        1
HAVING 
        COUNT(AppointmentDAY) = (SELECT 
                                        MAX(T1.daycount) 
                                FROM 
                                        (SELECT 
                                                AppointmentDAY, COUNT(AppointmentDAY) as daycount
                                        FROM 
                                                patients
                                        GROUP BY 
                                                AppointmentDAY) as T1)


 * sqlite:///jupyter_sql_tutorial.db
Done.


AppointmentDay,number_of_appointments
2016-06-06 00:00:00,4529


TASK 5: Calculate the monthly average number of appointments in the given dataset.
#### Hint : Use 'DATE_FORMAT()' function.

In [54]:
%%sql 

SELECT 
            STRFTIME('%Y-%m', AppointmentDay) AS year_and_month, AVG(STRFTIME('%Y-%m', AppointmentDay)) as monthly_avg
FROM 
            patients
GROUP BY 
            1

 * sqlite:///jupyter_sql_tutorial.db
Done.


year_and_month,monthly_avg
2016-04,2016.0
2016-05,2016.0
2016-06,2016.0


TASK 6: Find the month with the highest number of appointments in the given dataset.

In [57]:
%%sql

SELECT 
        STRFTIME('%Y-%m', AppointmentDay) as month, 
        COUNT(AppointmentDAY) as appointment_no
FROM 
        patients
GROUP BY 
        1
HAVING 
        COUNT(AppointmentDAY) = (SELECT 
                                        MAX(T1.daycount) 
                                FROM 
                                        (SELECT 
                                                STRFTIME('%Y-%m', AppointmentDay) AS year_and_month, 
                                                COUNT(AppointmentDAY) as daycount
                                        FROM 
                                                patients
                                        GROUP BY 
                                                year_and_month) as T1)

 * sqlite:///jupyter_sql_tutorial.db
Done.


month,appointment_no
2016-05,78202


TASK 7: Calculate the weekly average number of appointments in the given dataset.

In [58]:
 %%sql 
 
 SELECT 
        STRFTIME('%Y', AppointmentDay) AS year, 
        STRFTIME('%m', AppointmentDay) AS week, 
        COUNT(AppointmentDAY) as week_average
 FROM 
        patients
 GROUP BY 
        1,2

 * sqlite:///jupyter_sql_tutorial.db
Done.


year,week,week_average
2016,04,3104
2016,05,78202
2016,06,25682


TASK 8: Find the week with the highest number of appointments in the given dataset.

In [65]:
%%sql

SELECT 
        STRFTIME('%Y', AppointmentDay) as year, 
        STRFTIME('%w', AppointmentDay) as week, 
        COUNT(AppointmentDAY) as appointment_no
FROM 
        patients
GROUP BY 
        1,2
HAVING 
        COUNT(AppointmentDAY) = (SELECT 
                                        MAX(T1.daycount) 
                                FROM 
                                        (SELECT 
                                                STRFTIME('%w', AppointmentDay) as week, 
                                                COUNT(AppointmentDAY) as daycount
                                        FROM 
                                                patients
                                        GROUP BY 
                                                1) as T1)
      
      
      
      


 * sqlite:///jupyter_sql_tutorial.db
Done.


year,week,appointment_no
2016,3,25090


TASK 9: What is the distribution of appointments based on gender in the dataset?

In [68]:
%%sql

SELECT 
        Gender, count(Gender) as gender_count
FROM 
        patients
GROUP BY 
        1


 * sqlite:///jupyter_sql_tutorial.db
Done.


Gender,gender_count
F,70119
M,36869


TASK 10: Calculate the number of appointments per weekday in the given dataset. Order the appointment counts in descending.
#### Hint : Use 'DAYNAME()' function.


In [64]:
%%sql 

SELECT 
        case cast(strftime('%w', AppointmentDay) as integer)
           when 0 then 'Sunday'
           when 1 then 'Monday'
           when 2 then 'Tuesday'
           when 3 then 'Wednesday'
           when 4 then 'Thursday'
           when 5 then 'Friday'
           else 'Saturday' end as day,
        count(*) as day_count
FROM 
        patients
GROUP BY 
        1
ORDER BY 
        2 DESC

 * sqlite:///jupyter_sql_tutorial.db
Done.


day,day_count
Wednesday,25090
Tuesday,24831
Monday,21904
Friday,18465
Thursday,16659
Saturday,39


TASK 11: Calculate the average time between scheduling and the appointment day in the given dataset. Set to nearest whole number

In [67]:
%%sql

SELECT 
        ROUND(AVG(T1.time_diff),0) as avg_time
FROM  
        (SELECT 
                julianday(AppointmentDay) -  julianday(ScheduledDay) as time_diff
		FROM 
                patients) as T1

 * sqlite:///jupyter_sql_tutorial.db
Done.


avg_time
10.0
